In [ ]:
!pip install huggingface_hub

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from huggingface_hub import login
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, Input, Model
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.optim as optim
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset
from torchvision import datasets, transforms
from torchvision.transforms import functional as TF
from datasets import load_dataset
from PIL import Image
import pandas as pd
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm



In [ ]:
ds = load_dataset("zh-plus/tiny-imagenet")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

(…)-00000-of-00001-1359597a978bc4fa.parquet:   0%|          | 0.00/146M [00:00<?, ?B/s]

(…)-00000-of-00001-70d52db3c749a935.parquet:   0%|          | 0.00/14.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
ds.shape

{'train': (100000, 2), 'valid': (10000, 2)}

In [ ]:
transform_train_rgb = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224),
    transforms.RandomApply([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    ], p=0.5),  # Áp dụng các biến đổi này với xác suất 0.5
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_test_rgb = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Định nghĩa các biến đổi cho ảnh grayscale
transform_train_gray = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0) == 1 else x),  # Chuyển đổi sang RGB nếu cần
])

transform_test_gray = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Grayscale(num_output_channels=3),  # Chuyển đổi thành RGB
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# Định nghĩa Dataset cho ảnh RGB và grayscale
class ImageNetHardDataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]['image'].convert('RGB')  # Chuyển đổi thành RGB
        label = self.data[idx]['label']
        if self.transform:
            image = self.transform(image)
        return image, label


In [ ]:
# Chia tập dữ liệu thành train và test
train_size = int(0.8 * len(ds['train']))
test_size = len(ds['train']) - train_size
train_data, test_data = random_split(ds['train'], [train_size, test_size])
# Tạo Dataset cho ảnh RGB và grayscale
train_dataset_rgb = ImageNetHardDataset(train_data, transform_train_rgb)
train_dataset_gray = ImageNetHardDataset(train_data, transform_train_gray)
test_dataset_rgb = ImageNetHardDataset(test_data, transform_test_rgb)
test_dataset_gray = ImageNetHardDataset(test_data, transform_test_gray)


# Kết hợp Dataset bằng ConcatDataset
train_dataset = ConcatDataset([train_dataset_rgb, train_dataset_gray])
test_dataset = ConcatDataset([test_dataset_rgb, test_dataset_gray])

# Tạo DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes=200):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, num_classes),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
num_classes = 0
all_labels = []
for _, labels in train_loader:
    all_labels.extend(labels.tolist())
print(f"Max label: {max(all_labels)}, Num classes: {num_classes}")
num_classes = max(all_labels) + 1

In [ ]:
# Thông số cho mô hình
input_dim =  3*224 * 224  # Ảnh RGB đã được resize về 224x224
hidden_dim = 1024           # Số chiều của tầng ẩn           # Số lớp trong dữ liệu
learning_rate = 0.001
num_classes= 202
num_epochs = 8

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Khởi tạo mô hình
model = MLP(input_dim=input_dim, hidden_dim=hidden_dim, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-3)


In [ ]:
def train(model, loader, optimizer, criterion, device):
    model.train()
    total_loss, correct = 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Tính toán kết quả
        total_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()

    avg_loss = total_loss / len(loader.dataset)
    accuracy = 100 * correct / len(loader.dataset)
    return avg_loss, accuracy


def validate(model, loader, criterion, device):
    model.eval()
    total_loss, correct = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            # Forward
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Tính toán kết quả
            total_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()

    avg_loss = total_loss / len(loader.dataset)
    accuracy = 100 * correct / len(loader.dataset)
    return avg_loss, accuracy


In [ ]:
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = validate(model, test_loader, criterion, device)

    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%")

Epoch [1/5]
Train Loss: 5.3722, Train Accuracy: 0.47%
Validation Loss: 5.3005, Validation Accuracy: 0.43%
Epoch [2/5]
Train Loss: 5.3003, Train Accuracy: 0.48%
Validation Loss: 5.3002, Validation Accuracy: 0.39%


In [ ]:
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = validate(model, test_loader, criterion, device)

    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%")

Epoch [1/5]
Train Loss: 5.3507, Train Accuracy: 0.48%
Validation Loss: 5.3007, Validation Accuracy: 0.49%
Epoch [2/5]
Train Loss: 5.2998, Train Accuracy: 0.51%
Validation Loss: 5.3007, Validation Accuracy: 0.46%
Epoch [3/5]
Train Loss: 5.2991, Train Accuracy: 0.48%
Validation Loss: 5.3007, Validation Accuracy: 0.47%
Epoch [4/5]
Train Loss: 5.2998, Train Accuracy: 0.47%
Validation Loss: 5.3006, Validation Accuracy: 0.40%
Epoch [5/5]
Train Loss: 5.2989, Train Accuracy: 0.49%
Validation Loss: 5.3006, Validation Accuracy: 0.40%


In [ ]:
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = validate(model, test_loader, criterion, device)

    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%")

Epoch [1/5]
Train Loss: 4.9022, Train Accuracy: 3.63%
Validation Loss: 4.9264, Validation Accuracy: 4.41%
Epoch [2/5]
Train Loss: 4.7427, Train Accuracy: 5.31%
Validation Loss: 4.8687, Validation Accuracy: 5.48%
Epoch [3/5]
Train Loss: 4.6695, Train Accuracy: 6.03%
Validation Loss: 4.8324, Validation Accuracy: 6.15%


In [ ]:
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = validate(model, test_loader, criterion, device)

    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%")

Epoch [1/8]
Train Loss: 4.9824, Train Accuracy: 3.25%
Validation Loss: 4.9807, Validation Accuracy: 4.14%
Epoch [2/8]
Train Loss: 4.8016, Train Accuracy: 4.75%
Validation Loss: 4.8962, Validation Accuracy: 5.35%
Epoch [3/8]
Train Loss: 4.7195, Train Accuracy: 5.53%
Validation Loss: 4.8558, Validation Accuracy: 5.95%
Epoch [4/8]
Train Loss: 4.6628, Train Accuracy: 6.15%
Validation Loss: 4.8227, Validation Accuracy: 6.37%
Epoch [5/8]
Train Loss: 4.6145, Train Accuracy: 6.75%
Validation Loss: 4.8032, Validation Accuracy: 6.80%
Epoch [6/8]
Train Loss: 4.5757, Train Accuracy: 7.18%
Validation Loss: 4.7522, Validation Accuracy: 7.22%
Epoch [7/8]
Train Loss: 4.5448, Train Accuracy: 7.55%
Validation Loss: 4.7429, Validation Accuracy: 7.60%
